<a href="https://colab.research.google.com/github/aditya0811/PyTorch/blob/master/flower_data_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!wget "https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip" 



--2019-01-03 17:01:33--  https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.1.254
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.1.254|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 311442766 (297M) [application/zip]
Saving to: ‘flower_data.zip.1’

flower_data.zip.1   100%[===================>] 297.01M  92.4MB/s    in 3.4s    

2019-01-03 17:01:36 (88.4 MB/s) - ‘flower_data.zip.1’ saved [311442766/311442766]



In [0]:
!unzip "flower_data.zip"

Archive:  flower_data.zip
replace flower_data/valid/61/image_06296.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [0]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import transforms, models,datasets
import torchvision.models as models
from matplotlib.ticker import FormatStrFormatter
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

data_dir = "/content/flower_data"



# Number of classes in the dataset
num_classes = 102

# Batch size for training (change depending on how much memory you have)
batch_size = 32

# Number of epochs to train for 
num_epochs = 30

# Flag for feature extracting. When False, we finetune the whole model, 
#   when True we only update the reshaped layer params
feature_extract = False


In [0]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [0]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [0]:
#TRAINING MODEL
import torchvision.models as models
densenet = models.densenet161(pretrained=True)



/usr/local/lib/python3.6/dist-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [0]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    
   

   
    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes) 
        input_size = 224

    
    else:
        print("Invalid model name, exiting...")
        exit()
    
    return model_ft, input_size


model_ft, input_size = initialize_model('densenet', num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
print(model_ft)

/usr/local/lib/python3.6/dist-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [0]:
import os


In [0]:

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(30),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'valid']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'valid']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Initializing Datasets and Dataloaders...


In [0]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are 
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 features.conv0.weight
	 features.norm0.weight
	 features.norm0.bias
	 features.denseblock1.denselayer1.norm1.weight
	 features.denseblock1.denselayer1.norm1.bias
	 features.denseblock1.denselayer1.conv1.weight
	 features.denseblock1.denselayer1.norm2.weight
	 features.denseblock1.denselayer1.norm2.bias
	 features.denseblock1.denselayer1.conv2.weight
	 features.denseblock1.denselayer2.norm1.weight
	 features.denseblock1.denselayer2.norm1.bias
	 features.denseblock1.denselayer2.conv1.weight
	 features.denseblock1.denselayer2.norm2.weight
	 features.denseblock1.denselayer2.norm2.bias
	 features.denseblock1.denselayer2.conv2.weight
	 features.denseblock1.denselayer3.norm1.weight
	 features.denseblock1.denselayer3.norm1.bias
	 features.denseblock1.denselayer3.conv1.weight
	 features.denseblock1.denselayer3.norm2.weight
	 features.denseblock1.denselayer3.norm2.bias
	 features.denseblock1.denselayer3.conv2.weight
	 features.denseblock1.denselayer4.norm1.weight
	 features.de

In [0]:
print(dataloaders_dict)

{'train': <torch.utils.data.dataloader.DataLoader object at 0x7f9792624be0>, 'valid': <torch.utils.data.dataloader.DataLoader object at 0x7f975e173b00>}


In [0]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'valid':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [0]:
import time
import copy
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs)



Epoch 0/14
----------
train Loss: 1.6073 Acc: 0.7099
valid Loss: 0.7628 Acc: 0.8582

Epoch 1/14
----------
train Loss: 0.9563 Acc: 0.8358
valid Loss: 0.4283 Acc: 0.9425

Epoch 2/14
----------
train Loss: 0.6463 Acc: 0.8927
valid Loss: 0.2855 Acc: 0.9511

Epoch 3/14
----------
train Loss: 0.4911 Acc: 0.9147
valid Loss: 0.2205 Acc: 0.9645

Epoch 4/14
----------
train Loss: 0.4158 Acc: 0.9216
valid Loss: 0.1786 Acc: 0.9743

Epoch 5/14
----------
train Loss: 0.3605 Acc: 0.9322
valid Loss: 0.1532 Acc: 0.9743

Epoch 6/14
----------
train Loss: 0.3111 Acc: 0.9426
valid Loss: 0.1415 Acc: 0.9719

Epoch 7/14
----------
train Loss: 0.2685 Acc: 0.9522
valid Loss: 0.1151 Acc: 0.9792

Epoch 8/14
----------
train Loss: 0.2354 Acc: 0.9579
valid Loss: 0.1171 Acc: 0.9780

Epoch 9/14
----------
train Loss: 0.2200 Acc: 0.9597
valid Loss: 0.1043 Acc: 0.9829

Epoch 10/14
----------
train Loss: 0.2016 Acc: 0.9635
valid Loss: 0.1016 Acc: 0.9841

Epoch 11/14
----------
train Loss: 0.1978 Acc: 0.9609
valid Loss

In [0]:
torch.save({"state_dict":model_ft.state_dict()},'save1.pt')

In [0]:
model_ft.load_state_dict(torch.load('save1.pt'))

In [0]:
import time
import copy
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs)

Epoch 0/29
----------
train Loss: 0.1888 Acc: 0.9625
valid Loss: 0.0967 Acc: 0.9841

Epoch 1/29
----------
train Loss: 0.1910 Acc: 0.9611
valid Loss: 0.0979 Acc: 0.9841

Epoch 2/29
----------
train Loss: 0.1741 Acc: 0.9625
valid Loss: 0.0883 Acc: 0.9792

Epoch 3/29
----------
train Loss: 0.1691 Acc: 0.9661
valid Loss: 0.0939 Acc: 0.9792

Epoch 4/29
----------
train Loss: 0.1541 Acc: 0.9686
valid Loss: 0.0896 Acc: 0.9817

Epoch 5/29
----------
train Loss: 0.1681 Acc: 0.9628
valid Loss: 0.0847 Acc: 0.9878

Epoch 6/29
----------
train Loss: 0.1418 Acc: 0.9692
valid Loss: 0.0799 Acc: 0.9841

Epoch 7/29
----------
train Loss: 0.1441 Acc: 0.9687
valid Loss: 0.0804 Acc: 0.9841

Epoch 8/29
----------
train Loss: 0.1278 Acc: 0.9728
valid Loss: 0.0761 Acc: 0.9878

Epoch 9/29
----------
train Loss: 0.1257 Acc: 0.9733
valid Loss: 0.0735 Acc: 0.9841

Epoch 10/29
----------
train Loss: 0.1241 Acc: 0.9754
valid Loss: 0.0733 Acc: 0.9878

Epoch 11/29
----------
train Loss: 0.1243 Acc: 0.9727
valid Loss

In [0]:
torch.save({"state_dict":model_ft.state_dict()},'save2.pt')